<div style="display: flex; align-items: center;">
    <img src="https://github.com/nagelt/Teaching_Scripts/raw/9d9e29ecca4b04eaf7397938eacbf116d37ddc93/Images/TUBAF_Logo_blau.png" width="500" height="auto" height="auto" style="margin-right: 100px;" />
    <div>
        <p><strong>Prof. Dr. Thomas Nagel</strong></p>
        <p>Chair of Soil Mechanics and Foundation Engineering<br>Geotechnical Institute<br>Technische Universität Bergakademie Freiberg.</p>
        <p><a href="https://tu-freiberg.de/en/soilmechanics">https://tu-freiberg.de/en/soilmechanics</a></p>
    </div>
</div>

# Einmassenschwinger, Grundlagen

![oscillator1dof.png](https://github.com/nagelt/soil_dynamics/raw/master/Images/oscillator1dof.png)

## Hilfsfunktionen

Module laden

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets
from ipywidgets import interact
import plot_functions.plot_settings
%run plot_functions/single_mass_oscillator_interaction.ipynb

Umwandlung sin-cos-Darstellung in Amplitude-Phasenwinkel und zurück (nur zur Info und Überprüfung, wird nicht zur Lösung gebraucht)

In [ ]:
def sc2ap(s, c):
    a = np.sqrt(s**2 + c**2)
    p = np.arctan2(s, c)
    return [a, p]

In [ ]:
def ap2sc(a, p):
    s = a*np.sin(p)
    c = a*np.cos(p)
    return [s, c]

Umrechnung der physikalischen Parameter in die Schwingungsparameter (Standardform)

In [ ]:
def standard_form(m, c, k, F_c, F_s):
    if k<=0 or m<=0 or c<0:
        print("Warning: unphysical parameters for mechanical oscillations!")
    omega_n = np.sqrt(k/m)
    D = c/(2*np.sqrt(k*m))
    a_c = F_c/k
    a_s = F_s/k
    return [omega_n, D, a_c, a_s]

Eingeschwungenen Zustand (partikuläre Lösung) berechnen, inklusive Vergrößerungsfunktion und Phasendifferenz

In [ ]:
def steady_state_solution(omega, omega_n, D, a_c, a_s):
    beta = omega/omega_n
    if beta==1 and D==0:
        print("Info: no steady-state solution for resonant excitation of undamped oscillator!")
        V = 0 
        psi = 0
        u_c = 0
        u_s = 0
    else:        
        V2 = 1.0/( (1-beta**2)**2 + (2*beta*D)**2 )
        V = np.sqrt(V2)
        psi = np.arctan2(2*D*beta, 1-beta**2)
        u_c = V2*( (1-beta**2)*a_c - 2*D*beta*a_s )
        u_s = V2*( (1-beta**2)*a_s + 2*D*beta*a_c )
    return [u_c, u_s, V, psi]

Anpassen der Gesamtlösung an die Anfangsbedingungen

In [ ]:
def transient_solution(t_0, u_0, v_0):
    # presumes existence of uh(t), vh(t), up(t), vp(t) 
    uh_c1 = uh_1(t_0)
    uh_c2 = uh_2(t_0)
    vh_c1 = vh_1(t_0)
    vh_c2 = vh_2(t_0)
    up_0 = up(t_0)
    vp_0 = vp(t_0)
    # LGS Ax=b
    A = np.array([[uh_c1, uh_c2], [vh_c1, vh_c2]])
    b = np.array([u_0-up_0, v_0-vp_0])
    x = np.linalg.solve(A, b)
    C_1 = x[0]
    C_2 = x[1]
    return [C_1, C_2]

Berechnen des Logarithmischen Dekrements (wichtige Größe in der experimentellen Schwingungsmessung) 

In [ ]:
def log_decrement(t_0, t_N, u_0, u_N, N):
    LD = (1.0/N)*np.log(u_0/u_N)
    return LD

## Simulation 

#### Passen Sie die Parameter entsprechend an:

Freie ($F_c=F_s=0$), ungedämpfte ($c=0$) oder gedämpfte ($c>0$) Schwingung

Erzwungene ($\hat{F}>0$), ungedämpfte ($c=0$) oder gedämpfte ($c>0$) Schwingung

In [ ]:
m = 3.0   # kg
k = 10.0   # N/m
c = 1.0   # Ns/m   critical damping for c=2*sqrt(k*m)=10.95445115...
F_c = 0.0   # N
F_s = 5.0   # N
omega = 10.0   # rad/s
t_0 = 0.0   # s
u_0 = 0.1   # m
v_0 = 0.0   # m/s

Berechnen der Schwingungsparameter und der zur Kraftanregung äquivalenten Federfusspunktanregung

In [ ]:
[omega_n, D, a_c, a_s] = standard_form(m, c, k, F_c, F_s)   
print("omega_n = {:.3f} rad/s".format(omega_n), "   D = {:.3f}".format(D))

# define harmonic excitation as function a(t)
a = lambda t: a_c*np.cos(omega*t) + a_s*np.sin(omega*t)

Berechnen des eingeschwungenen Zustands (partikuläre Lösung)

In [ ]:
[u_c, u_s, V, psi] = steady_state_solution(omega, omega_n, D, a_c, a_s)
print("V = {:.3f}".format(V), "   psi = {:.3f} rad".format(psi))
print("u_c = {:.4f} m".format(u_c), "   u_s = {:.4f} m".format(u_s))

# define particular solution u_p(t) and its time derivative v_p(t)
if D==0 and omega==omega_n:
    up = lambda t: 0.5*a_c*omega*t*np.sin(omega*t) - 0.5*a_s*omega*t*np.cos(omega*t)
    vp = lambda t: 0.5*a_c*omega*(np.sin(omega*t) + t*omega*np.cos(omega*t)) - 0.5*a_s*omega*(np.cos(omega*t) - t*omega*np.sin(omega*t))
else:
    up = lambda t: u_c*np.cos(omega*t) + u_s*np.sin(omega*t)
    vp = lambda t: (-u_c*np.sin(omega*t) + u_s*np.cos(omega*t)) * omega

Anpassen der Gesamtlösung je nach Dämpfungsgrad (unter-/überkritisch) an die Anfangsbedingungen

In [ ]:
# define two solutions u_h(t) and their time derivative v_h(t), depending on discriminant
if D < 1:
    omega_d = omega_n*np.sqrt(1-D**2)
    delta = omega_n*D
    print("omega_d = {:.3f} rad/s".format(omega_d), "   delta = {:.3f} 1/s".format(delta))
    uh_1 = lambda t: np.exp(-delta*t)*np.cos(omega_d*t)
    uh_2 = lambda t: np.exp(-delta*t)*np.sin(omega_d*t)
    vh_1 = lambda t: np.exp(-delta*t)*( -delta*np.cos(omega_d*t) - omega_d*np.sin(omega_d*t) )
    vh_2 = lambda t: np.exp(-delta*t)*( -delta*np.sin(omega_d*t) + omega_d*np.cos(omega_d*t) )
elif D > 1:
    delta_1 = omega_n*(D - np.sqrt(D**2 - 1))
    delta_2 = omega_n*(D + np.sqrt(D**2 - 1))
    print("delta_1 = {:.3f} 1/s".format(delta_1), "   delta_2 = {:.3f} 1/s".format(delta_2) )
    uh_1 = lambda t: np.exp(-delta_1*t)
    uh_2 = lambda t: np.exp(-delta_2*t)
    vh_1 = lambda t: np.exp(-delta_1*t)*(-delta_1)
    vh_2 = lambda t: np.exp(-delta_2*t)*(-delta_2)
else:
    delta = omega_n*D
    print("delta = {:.3f} 1/s".format(delta), "   (critical damping)")
    uh_1 = lambda t: np.exp(-delta*t)
    uh_2 = lambda t: t*np.exp(-delta*t)
    vh_1 = lambda t:-np.exp(-delta*t)*delta
    vh_2 = lambda t: np.exp(-delta*t) - t*np.exp(-delta*t)*delta
    
[C_1, C_2] = transient_solution(t_0, u_0, v_0)
print("C_1 = {:.3f} m".format(C_1), "C_2 = {:.3f} m".format(C_2))

# define adjusted solution of homogenous solution and total solution
uh = lambda t: C_1*uh_1(t) + C_2*uh_2(t) 
u = lambda t:  uh(t) + up(t) 

Erstellen diskreter Lösungsverläufe auf einem Zeitraster und grafische Darstellung

In [ ]:
Nplot=5
t_5P = t_0 + Nplot*2*np.pi/omega_n   # <Nplot> periods of free vibrations (undamped)
t_sample = np.linspace(t_0, t_5P , 250)   # discretization with 250 points in time
a_sample = a(t_sample)
uh_sample = uh(t_sample)
up_sample = up(t_sample)
u_sample = u(t_sample)

In [ ]:
plt.plot(t_sample, uh_sample, 'b--', t_sample, up_sample, 'b:', t_sample, u_sample, 'b');
plt.xlabel(r"$t$");
plt.ylabel(r"$u$");
plt.legend(["$u_h$","$u_p$", "$u$"]);

In [ ]:
plt.plot(t_sample, a_sample, 'r', t_sample, up_sample, 'b:');
plt.xlabel(r"$t$");
plt.ylabel(r"$a, u$");
plt.legend(["$a$","$u_p$"]);

**Diskussion:** Wie würden Sie das Einschalten einer Maschine simulieren, d.h. mit welchen Parametern und Anfangsbedingungen?

In [ ]:
interactive_tau_u_plot()   # nondimensional parameters

## Parameterbestimmung

### Ausschwingversuch

In [ ]:
t1 = 0   # s
t2 = 1.0   # s
u1 = 0.1   # m
u2 = 0.08   # m
N = 1

Was wir im Experiment sehen ist die gedämpfte Eigenfrequenz

In [ ]:
T_id = t2 - t1
omega_d_id = 2.0*np.pi/T_id

Nach einer Periode, wiederholen sich die harmonischen Funktionen, dadurch erfassen wir das Abklingen $e^{-\delta t}$ und nutzen $e^{-\delta t_2}=e^{-\delta(t_1+T)}=e^{-\delta t_1} e^{-\delta T}$

In [ ]:
LD = log_decrement(t1, t2, u1, u2, N)
delta_id = LD/T_id
c_id = 2*delta_id*m
print("c_id = {:.3f} Ns/m".format(c_id) )

Wir erinnern uns an $\omega_d=\omega_n\sqrt{1-D^2}$ und $\delta=D \omega_n$

In [ ]:
omega_n_id_squared = omega_d_id**2 + delta_id**2
k_id = omega_n_id_squared*m
print("k_id = {:.3f} N/m".format(k_id))

### Schwingsaitenwaage

In [ ]:
omega_measured = 10.0
D_measured = 0.1

ungedämpfte Eigenkreisfrequenz $\omega_n=\sqrt{\frac{k}{m}}$

In [ ]:
omega_n_measured_1 = omega_measured
k_measured_1 = m*omega_n_measured_1**2    
print("k1 = {:.3f} N/m".format(k_measured_1) )

gedämpfte Eigenkreisfrequenz $\omega_d=\omega_n\sqrt{1-D^2}$

In [ ]:
omega_n_measured_2 = omega_measured/np.sqrt(1-D_measured**2)
k_measured_2 = m*omega_n_measured_2**2    
print("k2 = {:.3f} N/m".format(k_measured_2), "   Delta = {:2.2f} %".format(100*np.abs(k_measured_2-k_measured_1)/k_measured_1))

maximale Antwortamplitude bei der Anregungsfrequenz $\omega_\mathrm{MA}=\omega_n\sqrt{1-2 D^2}$

In [ ]:
omega_n_measured_3 = omega_measured/np.sqrt(1-2*D_measured**2)
k_measured_3 = m*omega_n_measured_3**2    
print("k3 = {:.3f} N/m".format(k_measured_3), "   Delta = {:2.2f} %".format(100*np.abs(k_measured_3-k_measured_1)/k_measured_1))

**TODO** Energiebilanz (kinetisch, potentiell) über einen Zyklus 